In [2]:
import numpy as np
import pandas as pd
import redis

In [3]:
hostname = "127.0.0.1"
port = 6379
r = redis.StrictRedis(hostname, port)

#### Load the data (numpy zip)

In [6]:
# To have this file, run file face-recognition-system first
np_file = np.load('dataframe_students_teacher.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ..."
1,Barack Obama,teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613..."
2,Chris Evans,student,"[0.72002447, -1.4682528, -0.77743137, -1.45999..."
3,Morgan Freeman,teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597..."
4,Scarlett Johansson,student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732..."


In [7]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie@student
1,Barack Obama,teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama@teacher
2,Chris Evans,student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans@student
3,Morgan Freeman,teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman@teacher
4,Scarlett Johansson,student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson@student


In [9]:
records = df[['name_role', 'Facial_Features']].to_dict(orient='records')
len(records)

5

#### Save data in Redis

In [12]:
for record in records:
  name_role = record['name_role']
  vector = record['Facial_Features']

  # convert numpy array into bytes
  vector_bytes = vector.tobytes()
  r.hset(name='academy:register', key=name_role, value=vector_bytes)